<a href="https://colab.research.google.com/github/HyeonhoonLee/Colab/blob/master/NIPA2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf


In [2]:
seed = 1234

In [3]:
data_path = '/content/drive/My Drive/DataCollection/NIPA2020/'

In [4]:
# Data loading
train_df = pd.read_csv('/content/drive/My Drive/DataCollection/NIPA2020/train.tsv', sep='\t', names= ["file_name", "Plant", "Disease"])
train_df.head()

,file_name,Plant,Disease
0,3_5_1123.jpg,3,5
1,3_20_1048.jpg,3,20
2,4_2_401.jpg,4,2
3,4_7_740.jpg,4,7
4,4_11_93.jpg,4,11


In [5]:
train_df["Plant"].value_counts()

13    6400
4     2400
7     1600
3     1600
8     1600
5      800
11     800
10     800
Name: Plant, dtype: int64

In [30]:
# plantclasss = {7:13, 1:4, 4:7, 3:3, 0:8, 6:5, 5:11, 2:10}

In [32]:
train_df.Plant = pd.factorize(train_df.Plant)[0]

In [33]:
train_df["Plant"].value_counts()

7    6400
1    2400
4    1600
3    1600
0    1600
6     800
5     800
2     800
Name: Plant, dtype: int64

In [6]:
train_df["Disease"].value_counts()

20    3200
6     1600
9     1600
1     1600
15     800
7      800
14     800
5      800
11     800
18     800
2      800
17     800
16     800
8      800
Name: Disease, dtype: int64

In [34]:
train_df.Disease = pd.factorize(train_df.Disease)[0]

In [ ]:
# diseaseclasss = {1:20, 7:6, 6:9, 8:1, 13:15, 5:7, 12:14, 4:5, 11:11, 3:18, 10:2, 2:17, 9:16, 0:8}

In [35]:
train_df["Disease"].value_counts()

1     3200
7     1600
6     1600
8     1600
13     800
5      800
12     800
4      800
11     800
3      800
10     800
2      800
9      800
0      800
Name: Disease, dtype: int64

In [36]:
# train_df['Plant'] = train_df['Plant'].astype(str)
# train_df['Disease'] = train_df['Disease'].astype(str)

In [37]:
# train_df['label'] = list(zip(train_df.Plant, train_df.Disease))
# train_df.head()

In [38]:
# train_p_df = train_df.drop(['Disease'], axis=1)
# train_p_df.tail()

In [39]:
# train_d_df = train_df.drop(['Plant'], axis=1)
# train_d_df.head()

In [40]:
# Data loading
test_df = pd.read_csv('/content/drive/My Drive/DataCollection/NIPA2020/test.tsv', sep='\t', names= ["file_name"])
test_df.head()

,file_name
0,0.jpg
1,1.jpg
2,2.jpg
3,3.jpg
4,4.jpg


In [41]:
from keras_preprocessing.image import ImageDataGenerator

In [42]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers, Model

In [43]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [44]:
train_generator=train_datagen.flow_from_dataframe(
  dataframe=train_df,
  directory="/content/drive/My Drive/DataCollection/NIPA2020/train",
  x_col="file_name",
  y_col=["Plant","Disease"],
  subset="training",
  batch_size=32,
  seed=seed,
  shuffle=True,
  class_mode="raw",
  target_size=(128,128))

Found 12800 validated image filenames.


In [45]:
validation_generator=train_datagen.flow_from_dataframe(
  dataframe=train_df,
  directory="/content/drive/My Drive/DataCollection/NIPA2020/train",
  x_col="file_name",
  y_col=["Plant","Disease"],
  subset="validation",
  batch_size=32,
  seed=seed,
  shuffle=True,
  class_mode="raw",
  target_size=(128,128))

Found 3200 validated image filenames.


In [46]:
# p_model = tf.keras.applications.ResNet152(
    # include_top=True, weights='imagenet', input_shape=(128,128,3), classes=18)

In [47]:
# model = tf.keras.Sequential()
# model.add(Conv2D(64, (3, 3), padding='same',
#                  input_shape=(128,128,3)))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(32, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(18, activation='softmax'))

In [48]:
inp = Input(shape = (128,128,3))
x = Conv2D(32, (3, 3), padding = 'same')(inp)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding = 'same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output1 = Dense(8, activation = 'sigmoid')(x)
output2 = Dense(14, activation = 'sigmoid')(x)
model = Model(inp,[output1,output2])


In [49]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 128, 128, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 128, 128, 32) 0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 126, 126, 32) 9248        activation_5[0][0]               
_______________________________________________________________________________________

In [50]:
# model.compile(optimizers.rmsprop(lr = 0.0001, decay = 1e-6),
# loss = ["binary_crossentropy","binary_crossentropy",,metrics = ["accuracy"])

In [56]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False)
# loss = tf.keras.losses.CategoricalCrossentropy(
#     from_logits=False)

In [57]:
model.compile(optimizer='adam', 
              # loss = {"dense_1" : loss, "dense_2" : loss},
              loss = [loss, loss],
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
              # metrics='mae'
              )

In [58]:
checkpoint_path = os.path.join(data_path, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

/content/drive/My Drive/DataCollection/NIPA2020 -- Folder already exists 



In [59]:
# from sklearn.preprocessing import MultiLabelBinarizer
# # Fit a MultiLabelBinarizer
# mlb = MultiLabelBinarizer()
# mlb.fit(train_df['label'].values.tolist())
# # The helper function
# def multilabel_flow_from_dataframe(data_generator, mlb):
#     for x, y in data_generator:
#         indices = y.astype(np.int).tolist()
#         y_multi = mlb.transform(
#             train_df.iloc[indices]['label'].values.tolist()
#         )
#         yield x, y_multi

In [60]:
# multilabel_generator = multilabel_flow_from_dataframe(
#     train_generator, mlb
# )
# multilabel_generator2 = multilabel_flow_from_dataframe(
#     validation_generator, mlb
# )

In [61]:
# model.fit(multilabel_generator,
#           # steps_per_epoch=STEP_SIZE_TRAIN,
#           validation_data=multilabel_generator2,
#           validation_steps=10,
#           epochs=1,verbose=2
# )

In [62]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(2)])
# def generator_wrapper2(generator):
#     for batch_x,batch_y in generator:
#         yield (batch_x,[batch_y[:,i] for i in range(1)])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=generator_wrapper(train_generator),
                    steps_per_epoch=STEP_SIZE_TRAIN,
                  validation_data=generator_wrapper(validation_generator),
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,verbose=1
)

Epoch 1/10
130/400 [========>.....................] - ETA: 12:38 - loss: 3.7310 - dense_4_loss: 1.4891 - dense_5_loss: 2.2419 - dense_4_sparse_categorical_accuracy: 0.4173 - dense_5_sparse_categorical_accuracy: 0.2171

In [ ]:
# def join_generators(generators):
#     while True: # keras requires all generators to be infinite
#         data = [next(g) for g in generators]

#         x = [d[0] for d in data ]
#         y = [d[1] for d in data ]

#         yield x, y

In [ ]:
# STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
# STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
# # STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
# model.fit(join_generators(train_generator),
#           # steps_per_epoch=STEP_SIZE_TRAIN,
#           epochs=10,
#           validation_data=join_generators(validation_generator),
#           # validation_steps=STEP_SIZE_VALID,
#           validation_steps=1
# )

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
model.fit(train_generator,
          # steps_per_epoch=STEP_SIZE_TRAIN,
          epochs=10,
          validation_data=validation_generator,
          # validation_steps=STEP_SIZE_VALID,
          validation_steps=1
)

In [ ]:
# plantclasss = {7:13, 1:4, 4:7, 3:3, 0:8, 6:5, 5:11, 2:10}
# diseaseclasss = {1:20, 7:6, 6:9, 8:1, 13:15, 5:7, 12:14, 4:5, 11:11, 3:18, 10:2, 2:17, 9:16, 0:8}